In [1]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
import xlrd
# Read the data file we got through data cleaning. 
wb = xlrd.open_workbook('BikeRentCount.xlsx')
# The file consists of five sheets and each sheet contains data for one station.
sheets = wb.sheet_names()
MaxRentingRate=[]
RentingRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeRentCount.xlsx', sheet_name=sheets[i])
    #Calculate the average number of rented bikes for each twenty minutes of each station.
    rentingRates = data.mean()
    RentingRate.append(rentingRates)
    # Find the maximum of these rates that would be used for function NSPP
    maxRentingRate = max(rentingRates)
    MaxRentingRate.append(maxRentingRate)
    
# Example: the number of rented bikes at Station 2 for every 20 minutes (indexing from 0) 
RentingRate[1]

7:00-7:20     1.869565
7:20-7:40     4.967391
7:40-8:00     7.923913
8:00-8:20     4.619565
8:20-8:40     6.086957
8:40-9:00     4.717391
9:00-9:20     3.478261
9:20-9:40     3.217391
9:40-10:00    1.706522
dtype: float64

In [3]:
# Example: the number of rented bikes at Station 2(indexing from 0) during time 8:40-9:00
RentingRate[1][5]

4.717391304347826

In [4]:
# Example: the maximum number of rented bikes at station 2(indexing from 0)
MaxRentingRate[1]

7.923913043478261

In [5]:
# Similar data processing for 'BikeReturnCount.xlsx'
wb = xlrd.open_workbook('BikeReturnCount.xlsx')
sheets = wb.sheet_names()
MaxReturnRate=[]
ReturnRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeReturnCount.xlsx', sheet_name=sheets[i])
    returnRates = data.mean()
    ReturnRate.append(returnRates)
    maxReturnRate = max(returnRates)
    MaxReturnRate.append(maxReturnRate)

# Simulation constants

In [20]:
order_threshold = 2.0 #schedule the "rebalancing" once the number of bikes is less than the threshold
order_up_to = 5.0 # the number of bikes in the station would reach 5 after rebalancing
delivery_delay = 20 # it takes 20 minutes for bikes to be transported to the station for bike rebalancing
SIM_RUN = 100 #number of simulation runs
operation_cost = 1 # USD per bike for operation
oil_gas = 2 # USD per 1 refillment
service_fee = 5 # USD per bike per ride
PENALTY = 1 # CAD for cost of loss of business oportunity
loss_profit = 0.01 # CAD per bike per minute for loss of business opportunity
RunLength = 180 # 3 hours

In [7]:
def Rental(**kwargs):
    global revenue, penalty, Loss_profit
    #Check which station the rental event occurs in
    station_id = kwargs['stationID']
    #Use NSPP to schedule the next bike rental event for current station
    interarrival = NSPP(station_id,MaxRentingRate,RentingRate)
    SimFunctions.Schedule(Calendar,"Rent a bike",interarrival,stationID = station_id)
    # Per bike per minute for loss of business opportunity
    #Loss_profit += loss_profit * Num_bikes[station_id] * interarrival
    # Check if there are bikes available and update number of bikes for the station.
    if Num_bikes[station_id]>0:
        Num_bikes[station_id]-=1
        revenue += service_fee
    else:
            penalty += PENALTY
            
    #Schedule the bike rebalancing event once the number of bikes is less than the threshold 
    """
    There are two conditions to be met for the rebalancing operation. Schedule the rebalancing operation 
    once the number of bikes is less than the threshold and the previous rebalance operation has ended. 
    """
    if Num_bikes[station_id] <= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = order_up_to - Num_bikes[station_id]
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,num_ordered=quantity[station_id])

In [8]:
def RideEnd(**kwargs):
    global  revenue, penalty, Loss_profit
    station_id = kwargs['stationID']
    ##Use NSPP to schedule the next end of ride for current station
    interarrival = NSPP(station_id,MaxReturnRate,ReturnRate)
    SimFunctions.Schedule(Calendar,"Return a bike",interarrival,stationID = station_id)
    #Loss_profit += loss_profit * Num_bikes[station_id] * interarrival
    # Check if there are empty racks to keep the bike.
    if Num_bikes[station_id] <= Num_docks[station_id]:
        Num_bikes[station_id] += 1
    # If there is no empty racks,the customer will get 30% refunded.
    else:
        penalty += 0.3*service_fee

In [9]:
def rebalancing(**kwargs):
    global revenue , cost, Loss_profit
    station_id = kwargs['stationID']
    #The number of bikes needed for bike rebalancing
    num_ordered = kwargs['num_ordered']
    #cost due to bike rebalanccing
    cost += (num_ordered * operation_cost) + oil_gas 
    #The number of bikes changes after the rebalancing operation
    Num_bikes[station_id] += num_ordered
    #Set the num_oredered to zero
    quantity[station_id] = 0

In [10]:
# Prevent index out of range

def PW_ArrRate(p,t,Rates):
    hour = int(t/20)
    if hour <= 8:
        return Rates[p][hour]
    else:
        return Rates[p][-1]


In [11]:
def NSPP(q,MaxRate,Rate):
    PossibleArrival = SimClasses.Clock + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    while SimRNG.Uniform(0, 1, 1) >= PW_ArrRate(q,PossibleArrival,Rate)/(MaxRate[q]):
        PossibleArrival = PossibleArrival + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    nspp = PossibleArrival - SimClasses.Clock
    return nspp

In [12]:
# Get all trial solution 
trial_solution = []
for a in range(5,15):
    for b in range(10,27):
        for c in range(5,11):
            for d in range(5,15):
                for e in range(5,15):
                    if (a+b+c+d+e)==70:
                        trial_solution.append([a,b,c,d,e])
len(trial_solution)

480

In [21]:
Calendar = SimClasses.EventCalendar()
TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

ZSimRNG = SimRNG.InitializeRNSeed()

#Create an 2d array to record the balance of each trial solution in each simulation run
output = np.zeros((SIM_RUN,len(trial_solution)))
for k in range(SIM_RUN):
    for j in range(len(trial_solution)):
        Num_bikes=[]
        #Apply the our model to each trial solution
        for i in range(0,5,1):
            Num_bikes.append(trial_solution[j][i])
        # the number of racks of each station 
        Num_docks=[15,27,11,15,15]
        #initial number of ordered bikes for rebalancing
        quantity=[0,0,0,0,0]
        revenue = 0
        cost = 0
        penalty = 0
        Loss_profit = 0

        SimClasses.Clock = 0.0
        SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
        for i in range(0,5,1):
            #Use NSPP to schedule the first arrival for each station.
            SimFunctions.Schedule(Calendar,"Rent a bike",NSPP(i,MaxRentingRate,RentingRate),stationID = i)
            SimFunctions.Schedule(Calendar,"Return a bike",NSPP(i,MaxReturnRate,ReturnRate),stationID = i) 
        SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == 'Rent a bike':
            Rental(**NextEvent.Kwargs)
        elif NextEvent.EventType == 'Return a bike':
            RideEnd(**NextEvent.Kwargs)
        elif NextEvent.EventType == 'rebalancing':
            rebalancing(**NextEvent.Kwargs)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            if NextEvent.EventType == 'Rent a bike':
                Rental(**NextEvent.Kwargs)
            elif NextEvent.EventType == 'Return a bike':
                RideEnd(**NextEvent.Kwargs)
            elif NextEvent.EventType == 'rebalancing':
                rebalancing(**NextEvent.Kwargs)
        balance = revenue - cost - penalty - Loss_profit
        output[k][j]= balance




In [22]:
output

array([[271.5, 405. , 299. , ..., 290. , 280. , 355. ],
       [374. , 330. , 274. , ..., 305. , 328.5, 292. ],
       [222. , 288.5, 302.5, ..., 340. , 305. , 335. ],
       ...,
       [322. , 305.5, 368.5, ..., 300.5, 323.5, 316.5],
       [347. , 282.5, 293. , ..., 362. , 325.5, 323.5],
       [288.5, 310. , 344. , ..., 320. , 318.5, 310. ]])

In [23]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_balance=np.mean(output,axis=0)
#Get the best solution 
trial_solution[np.argmax(Estimated_Expected_balance)]

[12, 26, 10, 9, 13]